## Initializes DLC project for ephys tracking. 

** Run locally to run dlc GUI for labelling images **

To run: 
1. Run Anaconda prompt as Administrator 
2. Activate DLC enviornment 
    ``` conda activate DLC-GPU ```
3. cd to project folder 

    ``` F: ``` * if not in F drive
    
    ``` cd F:\ClarkP30_Recordings ```
3. Open Jupyter Lab 
    ``` jupyter lab ```
4. Open ephys_tracking.ipynb found in ```F:\ClarkP30_Recordings\DLC```


### Import deeplabcut toolbox 

In [1]:
import deeplabcut 

### Setup Project details

In [4]:
# Setup Project 
task = 'linear_track' # Enter the name of your experiment Task
experimenter = 'Berkowitz' # Enter the name of the experimenter
project_path = 'D:/DLC_analysis/app_ps1/' 

### Set paths to video files used for generating your training set

In [2]:
#These are the paths to the videos you'll be using to pull images for training set
import glob as glob

vid_path = 'D:/DLC_analysis/app_ps1/Videos//'
video = glob.glob(vid_path+'*.avi') # Enter the paths of your videos OR FOLDER you want to grab frames from.

### For .mpg files, run the following in the terminal to convert to .avi
Several bugs exsist for .mpg files when running DLC. Easiest to just convert to default. 

Navigate to videos. 

``` cd D:/DLC_analysis/app_ps1/Videos// ```

Use ffmpeg to convert from .mpg to .avi. 

``` ffmpeg -i video_name.mpg -crf 17 video_name.avi ```

### Initialize your project directory. 

In [6]:
path_config_file = deeplabcut.create_new_project(task,experimenter,video,copy_videos=True) 

Project "C:\Users\schafferlab\Documents\GitHub\position_tracking\linear_track-Berkowitz-2021-11-15" already exists!


### Edit config file. 

* change bodyparts from default to red & green leds
* change skeleton labels
* set number of frames to use for training to 60
* set network type at resnet_101 

### Open config.yaml file, change parameters, save back to directory

In [8]:
import ruamel.yaml

config_path = 'd:\\Users\\BClarkLab\\GoogleDrive_\\DLC_analysis\\ephys_bitfields-Berkowitz-2021-02-09\\config.yaml'

yaml = ruamel.yaml.YAML()
# yaml.preserve_quotes = True
with open(config_path) as fp:
    data = yaml.load(fp)
# Update config file to track leds     
data['bodyparts'] = ['red_led','green_led']
data['skeleton'] =[['red_led','green_led']]
data['numframes2pick'] = 60
data['default_net_type'] = 'resnet_101'

with open(config_path, "w") as f:
    yaml.dump(data, f)


### Extract frames from videos for training

```extract_frames``` will ask if you want to extract frames for each video. yes to include frames from that video or no to skip.

In [ ]:
deeplabcut.extract_frames(config_path,mode = 'automatic',algo = 'kmeans', crop = False)

### Label Frames 

In [ ]:
deeplabcut.label_frames(config_path)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


### Create Training Dataset

In [ ]:
deeplabcut.create_training_dataset(config_path, augmenter_type='imgaug')

### Train Network

In [ ]:
deeplabcut.train_network(config_path, shuffle=1, displayiters=10,saveiters=500)

### Evaluate Network

In [ ]:
deeplabcut.evaluate_network(config_path)


### Analyze Videos

deeplabcut.analyze_videos(config_path,videofile_path, videotype=VideoType,save_as_csv=True,dynamic = (True,.1,50))